In [1]:
import pandas as pd
import os


import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import ftfy
import pickle
%matplotlib inline

import tensorflow as tf
import pandas as pd

os.chdir('D:\Project\Twitter_depression_detector\data')
os.getcwd()


'D:\\Project\\Twitter_depression_detector\\data'

In [2]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Lets see the model summary

In [3]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [4]:
df=pd.read_csv('CombinedTweets.csv')

In [5]:
df

,renderedContent,mentionedUserCount,label
0,fuck depression has hit killing eve rewatch ha...,0,1
1,@DREIDPIPER1 @_ReaalAmerican_ @realDonaldTrump...,4,1
2,What does Wisconsins weather and depression ha...,0,1
3,When u go out of town and ur bff cleans ur dep...,0,1
4,"wow i was so mad i meant to write ""what depres...",0,1
...,...,...,...
191679,"@zorawebs hellsyeah, but I have to study",1,0
191680,"@lakeeffectskid sounds good, wish i could too...",1,0
191681,@nhoustonreed happy birthday,1,0
191682,"@lilaleila Hehe, thanks sweetheart!",1,0


In [6]:
X = df.iloc[:,:-1]
y=df.iloc[:,-1]

In [7]:
X

,renderedContent,mentionedUserCount
0,fuck depression has hit killing eve rewatch ha...,0
1,@DREIDPIPER1 @_ReaalAmerican_ @realDonaldTrump...,4
2,What does Wisconsins weather and depression ha...,0
3,When u go out of town and ur bff cleans ur dep...,0
4,"wow i was so mad i meant to write ""what depres...",0
...,...,...
191679,"@zorawebs hellsyeah, but I have to study",1
191680,"@lakeeffectskid sounds good, wish i could too...",1
191681,@nhoustonreed happy birthday,1
191682,"@lilaleila Hehe, thanks sweetheart!",1


In [8]:
y

0         1
1         1
2         1
3         1
4         1
         ..
191679    0
191680    0
191681    0
191682    0
191683    0
Name: label, Length: 191684, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df, test_size = 0.20, random_state = 0)

In [10]:
train.head()

,renderedContent,mentionedUserCount,label
54684,"@grazia747 I agree with you,he could be on med...",1,1
172073,@SoniaAndTodd happy birthday Toddy,1,0
186406,"got back from swimming pool, on to shopping fo...",0,0
39891,Ironic how some are so quick to preach about b...,0,1
170375,@Muhsinah Yay. Can't wait to hear more from you,1,0


In [11]:
test.head()

,renderedContent,mentionedUserCount,label
84260,@MelissaAFrancis What started coronavirus is t...,1,1
26329,kinakanta ko yung alapaap napakanta na rin kai...,0,1
162560,just came from SFC's CLP Baptism.,0,0
42677,Why diverting to some vague criticism of Sande...,0,1
81633,Imagine how unemployment would feel without so...,0,1


In [12]:

def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')




In [13]:
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


In [14]:
DATA_COLUMN = 'renderedContent'
LABEL_COLUMN = 'label'


In [17]:

train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(16).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(16)

D:\Program Files (x86)\Anaconda\envs\twitter\lib\site-packages\transformers\tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)


Epoch 1/2
   3466/Unknown - 5431s 2s/step - loss: 0.0299 - accuracy: 0.9889

In [ ]:
pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']


In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

Num GPUs Available:  1


GPUs:  0


True

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True